In [541]:
import pandas as pd
import numpy as np
import matplotlib
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

Loading data as a dataframe.

In [542]:
san_francisco = pd.read_csv("./datasets/Flood_Health_Vulnerability.csv")
san_francisco = san_francisco[(np.abs(stats.zscore(san_francisco)) < 3).all(axis=1)] #Gets rid of outliers
san_francisco.head()

,Census Blockgroup,Children,Elderly,NonWhite,Poverty,Education,English,Elevation,SeaLevelRise,Precipitation,Diabetes,MentalHealth,Asthma,Disability,HousingViolations,Homeless,LivAlone,FloodHealthIndex,FloodHealthIndex_Quintiles
1,60750201001,0.079893,0.124,0.651000,0.509706,0.724000,0.232,7.83292,0.000000,0.020516,16.0115,62.0535,10.5741,0.238,10.24060,15.8921,0.522000,83.4860,5
3,60750178021,0.084215,0.088,0.584000,0.327362,0.843000,0.119,-5.96629,0.064579,0.024166,14.9856,54.0699,9.5802,0.229,7.04366,19.2683,0.427000,82.4925,5
6,60750201003,0.079893,0.124,0.651000,0.509706,0.724000,0.232,5.81684,0.000000,0.028468,15.9985,61.8836,10.5438,0.238,9.68738,3.2195,0.522000,81.5060,5
9,60750201002,0.079893,0.124,0.650999,0.509706,0.724001,0.232,13.91790,0.000000,0.013759,16.0125,62.0500,10.5693,0.238,11.57780,3.2195,0.522001,80.8327,5
15,60750176015,0.109834,0.141,0.733000,0.550862,0.806000,0.117,10.06420,0.000000,0.004025,16.0068,62.0689,10.5959,0.227,6.98324,19.2683,0.609000,79.1461,5


Basic Statistics

In [543]:
san_francisco.describe().drop("count", axis=0)

,Census Blockgroup,Children,Elderly,NonWhite,Poverty,Education,English,Elevation,SeaLevelRise,Precipitation,Diabetes,MentalHealth,Asthma,Disability,HousingViolations,Homeless,LivAlone,FloodHealthIndex,FloodHealthIndex_Quintiles
mean,6.075026e+10,0.130563,0.137482,0.544521,0.244480,0.886913,0.109916,134.64670,0.002717,0.005756,7.528631,18.097494,4.222185,0.094565,5.571640,3.726003,0.338921,47.167092,2.783567
std,1.127311e+05,0.049021,0.044716,0.205166,0.119968,0.086869,0.079790,97.14176,0.016904,0.008666,3.902217,13.803743,2.498270,0.037506,3.571443,4.710422,0.141094,11.772034,1.344799
min,6.075010e+10,0.009804,0.029776,0.128998,0.057373,0.593000,0.000000,-9.22917,0.000000,0.000000,0.000000,0.051100,0.000000,0.023084,0.000000,0.260200,0.064521,19.293200,1.000000
25%,6.075017e+10,0.089839,0.103000,0.386501,0.163074,0.828000,0.040500,54.76360,0.000000,0.000813,4.300000,8.633050,2.710900,0.074000,2.922535,0.520700,0.227000,38.602950,2.000000
50%,6.075025e+10,0.134989,0.139000,0.541000,0.225464,0.909000,0.100000,129.98200,0.000000,0.002516,6.400000,13.869000,3.651300,0.091000,4.764250,2.158900,0.317999,45.953100,3.000000
75%,6.075033e+10,0.170018,0.171000,0.675500,0.292193,0.958000,0.175000,194.00900,0.000000,0.006580,9.902050,21.476050,5.057700,0.111000,7.078370,4.598100,0.437000,54.033100,4.000000
max,6.075062e+10,0.295794,0.270000,0.981456,0.678177,1.000999,0.345000,433.66900,0.159217,0.048153,20.995600,63.750000,14.533600,0.238000,24.300000,19.268300,0.691000,83.486000,5.000000


Exploration with Flood Index Quintiles as a category

In [544]:
san_francisco.groupby("FloodHealthIndex_Quintiles").min()
from scipy.stats import spearmanr
from scipy.stats import pearsonr
columns = list(san_francisco.columns)[1:-2]
fhi = san_francisco[['FloodHealthIndex']]

correlationType  = dict()
significant_metrics = dict()
for column in columns:
    non_linear_corr, p1 = spearmanr(fhi, san_francisco[column])
    linear_corr, p2 = pearsonr(san_francisco[column], fhi)
    if abs(non_linear_corr) > abs(linear_corr[0]):
        larger_corr = non_linear_corr
        correlationType.update({column: "nonlinear"})
    else:
        larger_corr = linear_corr[0]
        correlationType.update({column: "linear"})
    
    if abs(larger_corr) >= .5:
        significant_metrics[column] = larger_corr

    
    print(f"{column}: {larger_corr} ({correlationType[column]})")
print(significant_metrics)
sig_column_list = list()
for key in significant_metrics:
    sig_column_list.append(key)
print(sig_column_list)

Children: -0.2857895487684717 (linear)
Elderly: -0.07932683793647982 (nonlinear)
NonWhite: 0.42777884442768427 (nonlinear)
Poverty: 0.6940349145675028 (linear)
Education: -0.5185598506634179 (linear)
English: 0.48954459987910764 (nonlinear)
Elevation: -0.6248862353664248 (linear)
SeaLevelRise: 0.20161734735146153 (linear)
Precipitation: 0.30381718961991616 (linear)
Diabetes: 0.5999321498483324 (linear)
MentalHealth: 0.633009433173236 (linear)
Asthma: 0.641611209836375 (linear)
Disability: 0.5759663051153814 (linear)
HousingViolations: 0.22729048797961451 (linear)
Homeless: 0.5297459087845907 (nonlinear)
LivAlone: 0.2508361644595812 (linear)
{'Poverty': 0.6940349145675028, 'Education': -0.5185598506634179, 'Elevation': -0.6248862353664248, 'Diabetes': 0.5999321498483324, 'MentalHealth': 0.633009433173236, 'Asthma': 0.641611209836375, 'Disability': 0.5759663051153814, 'Homeless': 0.5297459087845907}
['Poverty', 'Education', 'Elevation', 'Diabetes', 'MentalHealth', 'Asthma', 'Disability',

In [545]:
#x = san_francisco["Poverty"]
#y = san_francisco["FloodHealthIndex"]
#print(x.corr(y))
#a, b = np.polyfit(x, y, 1)
#model = np.poly1d(np.polyfit(x, y, 2))
#plt.scatter(x, y)
#plt.plot(x, a*x+b, color='purple') 
#plt.plot(x, model(x), color='green') 
#plt.ylabel("Flood Health Index")
#plt.xlabel("Poverty")
#plt.show()
#for i in san_francisco:
 #   san_francisco.plot(i, "FloodHealthIndex", kind="scatter")
  #  a, b = np.polyfit(san_francisco[i], y, 1)
   # plt.plot(san_francisco[i], a*san_francisco[i]+b, color='purple') 
    #plt.title(i)
    #print(f"Correlation of {i}: {san_francisco[i].corr(y)}")
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Load dataset
url = "./datasets/Flood_Health_Vulnerability.csv"
names = ["Census Blockgroup","Children","Elderly","NonWhite","Poverty","Education","English","Elevation","SeaLevelRise","Precipitation","Diabetes","MentalHealth","Asthma","Disability","HousingQuality","Homeless","LivAlone","FloodHealthIndex","FloodHealthIndex_Quintiles"
]
dataset = read_csv(url,names=names,header=0)
# Split-out validation dataset
array = dataset.values
X = san_francisco[sig_column_list] 
y = san_francisco[["FloodHealthIndex_Quintiles"]] 
for i in range(0, 100):
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)
    # Make predictions on validation dataset
    model = LinearDiscriminantAnalysis()
    model.fit(X_train, np.ravel(Y_train))
    predictions = model.predict(X_validation)
    # Evaluate predictions
    print(accuracy_score(Y_validation, predictions))
    
    #print(confusion_matrix(Y_validation, predictions))
    #print(classification_report(Y_validation, predictions))


0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
0.53
